In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import numpy as np
import pandas as pd

In [2]:
dataset = pd.read_csv('/Users/pierreperrin/Desktop/IG4/semestre7/Projet_DataScience/dataset_cleaned.csv',dtype={"content": str},low_memory=False)

In [3]:
dataset.head()

Message-ID                 Date  \
0  <18782981.1075855378110.JavaMail.evans@thyme>  2001-05-14 23:39:00   
1  <15464986.1075855378456.JavaMail.evans@thyme>  2001-05-04 20:51:00   
2  <24216240.1075855687451.JavaMail.evans@thyme>  2000-10-18 10:00:00   
3  <13505866.1075863688222.JavaMail.evans@thyme>  2000-10-23 13:13:00   
4  <30922949.1075863688243.JavaMail.evans@thyme>  2000-08-31 12:07:00   

                        From                         To    Subject X-cc X-bcc  \
0  'phillip.allen@enron.com'     'tim.belden@enron.com'        NaN  NaN   NaN   
1  'phillip.allen@enron.com'  'john.lavorato@enron.com'        Re:  NaN   NaN   
2  'phillip.allen@enron.com'   'leah.arsdall@enron.com'   Re: test  NaN   NaN   
3  'phillip.allen@enron.com'    'randall.gay@enron.com'        NaN  NaN   NaN   
4  'phillip.allen@enron.com'     'greg.piper@enron.com'  Re: Hello  NaN   NaN   

                                            X-Folder X-Origin  \
0  \Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Se...  Allen-P   
1  \Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Se...  Allen-P   
2    \Phillip_Allen_Dec2000\Notes Folders\'sent mail  Allen-P   
3    \Phillip_Allen_Dec2000\Notes Folders\'sent mail  Allen-P   
4    \Phillip_Allen_Dec2000\Notes Folders\'sent mail  Allen-P   

                                             content  ... Day  Month  Year  \
0                               here is our forecast  ...  14      5  2001   
1  traveling to have a business meeting takes the...  ...   4      5  2001   
2                      test successful  way to go     ...  18     10  2000   
3  randy  can you send me a schedule of the salar...  ...  23     10  2000   
4                   lets shoot for tuesday at 11 45   ...  31      8  2000   

   Moment  Meetings  Urgent  size_mail_quali  size_subject_quali  \
0       3         0       0                1                   1   
1       3         1       0                3                   1   
2       1         0       0                1                   1   
3       2         0       0                2                   1   
4       2         0       0                1                   1   

   Business_trip  hobby  
0              0      0  
1              1      1  
2              0      0  
3              0      0  
4              0      0  

[5 rows x 29 columns]

In [4]:
mail_content_brut=dataset['content']

In [5]:
import re

def clean_text(text):
    """
    Applies some pre-processing on the given text.

    Steps :
    - Removing HTML tags
    - Removing punctuation
    - Lowering text
    """
    
    # remove HTML tags
    text = re.sub(r'<.*?>', '', text)
    
    # remove the characters [\], ['] and ["]
    text = re.sub(r"\\", "", text)    
    text = re.sub(r"\'", "", text)    
    text = re.sub(r"\"", "", text)    
    
    # convert text to lowercase
    text = text.strip().lower()
    
    # replace punctuation characters with spaces
    filters='!"\'#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'
    translate_dict = dict((c, " ") for c in filters)
    translate_map = str.maketrans(translate_dict)
    text = text.translate(translate_map)

    return text

In [11]:
mail_content_cleaned = mail_content_brut

for i in range(len(mail_content_brut)):
    mail_content_cleaned[i] = clean_text(str(mail_content_brut[i]))

<ipython-input-11-940d37f807ac>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mail_content_cleaned[i] = clean_text(str(mail_content_brut[i]))


In [13]:
vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(mail_content_cleaned)

In [14]:
true_k = 2
model = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1)
model.fit(X)

KMeans(max_iter=100, n_clusters=2, n_init=1)

In [15]:
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()

In [20]:
for i in range(true_k):
    print("Cluster %d:" % i)
    for ind in order_centroids[i, :20]:
        print("%s" % terms[ind])

Cluster 0:
ect
enron
hou
com
20
2001
subject
2000
pm
jeff
thanks
know
cc
mail
message
ees
01
10
sent
meeting
Cluster 1:
00
description
chairperson
time
entry
migration
appointment
calendar
detailed
outlook
central
standard
team
date
pm
2001
30
mtg
11
sat


In [18]:
print("Prediction")
X = vectorizer.transform([mail_content_cleaned[23412]])
predicted = model.predict(X)
print(predicted)

Prediction
[0]


In [19]:
mail_content_cleaned[23412]

'it does work  we enjoyed seeing you  also  thanks for letting us crash temporarily at your house'